In [2]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("local[32]")
         .setAppName("My app")  
         .set("spark.executor.memory", "1g"))
sc = SparkContext(conf = conf)

In [3]:
sc.defaultParallelism

32

In [4]:
import pandas as pd
df = pd.read_csv("elawsCorpus-output.csv",encoding='latin-1')



In [5]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
sdf = sqlCtx.createDataFrame(df)
sdf.cache()

DataFrame[Unnamed: 0: bigint, Text: string, hyperlink: string]

In [6]:
import nltk
def process_law_text(input_text):
    text_law=input_text.replace("\x92","'").replace("\x96","-").replace('\x93','"').replace('\x94','"')
    tokens = nltk.word_tokenize(text_law)
    tokens_tagged = nltk.pos_tag(tokens)
#    tokens_tagged_nouns = list( filter(lambda x: x[1][0]=="N" and x[1][1]=="N",tokens_tagged))
#    nouns = list( map(lambda x: x[0], tokens_tagged_nouns) )
    nouns = list( map(lambda x: x[0], tokens_tagged) )


    nouns_lower= list(map(lambda x: x.lower(), nouns))
    nouns_unique = list(set(nouns_lower))
    nouns_unique.sort()
    return nouns_unique

In [7]:
ddf=sdf.rdd.map(lambda x: (x[0], process_law_text(x[1])))

0.14745473861694336


In [8]:
noun_lists=ddf.collect()



65.15251660346985


In [10]:
df1=pd.DataFrame(noun_lists,columns=["law","noun_list"])
#df1

In [11]:
df2 = pd.DataFrame(df1.noun_list.tolist(), index=df1.law)

In [12]:
df3=df2.stack().reset_index(level=1, drop=True).reset_index().rename(columns={0:"noun_list"})
df3["law"]=df3["law"].apply(lambda x: [x,])


In [13]:
df4=df3.groupby("noun_list").agg({"law":"sum"})

In [14]:
#df4
dfquery=df4.query('noun_list=="fax"')

In [15]:
dfquery

,law
noun_list,
fax,"[3, 7, 26, 45, 61, 82, 88, 90, 91, 97, 98, 101..."


In [16]:
dfquery.to_csv('query.csv')

In [17]:
cat query.csv

noun_list,law
fax,"[3, 7, 26, 45, 61, 82, 88, 90, 91, 97, 98, 101, 102, 116, 137, 147, 149, 161, 162, 184, 193, 199, 268, 293, 299, 304, 305, 317, 327, 347, 351, 352, 355, 363, 367, 369, 378, 393, 405, 414, 422, 471, 487, 571, 682]"


In [1]:
sc.stop()

NameError: name 'sc' is not defined

In [23]:
index=3-1
text_law=df.iloc[index][1]
tokens = nltk.word_tokenize(text_law)
text=nltk.Text(tokens)
text.concordance("fax")

Displaying 2 of 2 matches:
address of the person ; ( f ) sent by fax to the last known fax number of the p
; ( f ) sent by fax to the last known fax number of the person ; or ( g ) given


In [29]:
for j in dfquery.iloc[0][0]:
    text_law=df.iloc[j-1][1]
    tokens = nltk.word_tokenize(text_law)
    text=nltk.Text(tokens)
    print(df.iloc[j-1][2])
    text.concordance("fax")


https://www.ontario.ca/laws/statute/10r11
Displaying 2 of 2 matches:
address of the person ; ( f ) sent by fax to the last known fax number of the p
; ( f ) sent by fax to the last known fax number of the person ; or ( g ) given
https://www.ontario.ca/laws/statute/90b16
Displaying 6 of 6 matches:
ethods of endorsing and issuing 273.2 Fax filing 273.2 Refusal to endorse if co
orporation in default 273.3 Filing by fax 273.4 Electronic version prevails 273
luding telephone calls , voice mail , fax , e-mail , an automated touch-tone te
20 , Sched . 6 , s. 43 - not in force Fax filing 273.2 ( 1 ) Despite section 27
 . 6 , s. 43 - not in force Filing by fax 273.3 Despite any regulation made und
s and other documents may be filed by fax only with the Directors consent . 20
https://www.ontario.ca/laws/statute/90e06
Displaying 4 of 4 matches:
 courier or similar delivery method , fax or e-mail , to a returning office in 
 courier or similar delivery method , fax or e-mail , to the Chief Electo